# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import wandb
import os
from tqdm import tqdm


/home/blozerx/venvs/smai-env/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/blozerx/venvs/smai-env/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Ann

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## WANDB - https://wandb.ai/sudersha-sarraf-iiit-hyderabad/smai-a4-q2-colorization?nw=nwusersudershasarraf

In [4]:
#
# ---> Color Centroids Setup <--- #
#
def get_color_centroids(filepath="data/color_centroids.npy"):
    """
    Loads or generates the color centroids file.
    For reproducibility, if the file doesn't exist, it will be created using
    a fixed random seed.
    """
    if os.path.exists(filepath):
        print(f"Loading color centroids from {filepath}")
        return np.load(filepath)
    else:
        print(f"Centroids file not found. Generating a new one at {filepath}...")
        # This is a placeholder generation. In a real scenario, you'd run K-Means.
        # For consistency, we use a fixed seed.
        np.random.seed(42)
        centroids = np.random.randint(0, 256, size=(24, 3), dtype=np.uint8)
        np.save(filepath, centroids)
        print("Generated and saved new centroids.")
        return centroids

color_centroids = get_color_centroids()
# Convert to a tensor for use in the dataset
color_centroids_tensor = torch.from_numpy(color_centroids).float()

#
# ---> Custom Dataset for CIFAR-10 Colorization <--- #
#
class CIFAR10Colorization(Dataset):
    """
    Custom Dataset for CIFAR-10 Colorization.
    Input: Grayscale image (1, 32, 32)
    Target: Per-pixel color class map (32, 32)
    """
    def __init__(self, train=True, transform=None):
        self.cifar10 = datasets.CIFAR10(root='./data', train=train, download=True)
        self.centroids = color_centroids_tensor
        self.transform = transform
        self.grayscale_transform = transforms.Grayscale(num_output_channels=1)
        self.tensor_transform = transforms.ToTensor()

    def __len__(self):
        return len(self.cifar10)

    def __getitem__(self, idx):
        # Original image as a PIL Image
        img_pil, _ = self.cifar10[idx]
        
        # Apply transformations if any (e.g., for data augmentation)
        if self.transform:
            img_pil = self.transform(img_pil)

        # 1. Create the grayscale input for the model
        grayscale_img = self.grayscale_transform(img_pil)
        grayscale_tensor = self.tensor_transform(grayscale_img)

        # 2. Create the target label map
        img_rgb_tensor = self.tensor_transform(img_pil) * 255 # Scale to 0-255
        img_pixels = img_rgb_tensor.permute(1, 2, 0).view(-1, 3) # (1024, 3)

        # Calculate distance to each centroid and find the closest one
        # dists = torch.cdist(img_pixels, self.centroids) # Efficient but needs PyTorch 1.6+
        dists = torch.sqrt(torch.sum((img_pixels[:, None] - self.centroids[None, :])**2, dim=2))
        target_labels = torch.argmin(dists, dim=1) # (1024,)
        target_map = target_labels.view(32, 32) # (32, 32)

        return grayscale_tensor, target_map.long()

#
# ---> Model Architecture (Encoder-Decoder CNN) <--- #
#
class ColorizationNet(nn.Module):
    def __init__(self, nic=1, nf=32, nc=24):
        super(ColorizationNet, self).__init__()
        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Conv2d(nic, nf, kernel_size=3, padding=1),
            nn.BatchNorm2d(nf), nn.ReLU(True),
            nn.MaxPool2d(2, 2) # 32x32 -> 16x16
        )
        self.encoder2 = nn.Sequential(
            nn.Conv2d(nf, nf * 2, kernel_size=3, padding=1),
            nn.BatchNorm2d(nf * 2), nn.ReLU(True),
            nn.MaxPool2d(2, 2) # 16x16 -> 8x8
        )
        self.encoder3 = nn.Sequential(
            nn.Conv2d(nf * 2, nf * 4, kernel_size=3, padding=1),
            nn.BatchNorm2d(nf * 4), nn.ReLU(True),
            nn.MaxPool2d(2, 2) # 8x8 -> 4x4
        )

        # Decoder
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(nf * 4, nf * 2, kernel_size=2, stride=2), # 4x4 -> 8x8
            nn.BatchNorm2d(nf * 2), nn.ReLU(True)
        )
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(nf * 2, nf, kernel_size=2, stride=2), # 8x8 -> 16x16
            nn.BatchNorm2d(nf), nn.ReLU(True)
        )
        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(nf, nc, kernel_size=2, stride=2), # 16x16 -> 32x32
            nn.BatchNorm2d(nc), nn.ReLU(True)
        )

        # Classifier
        self.classifier = nn.Conv2d(nc, nc, kernel_size=1)

    def forward(self, x):
        # Encoder path
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)

        # Decoder path
        d1 = self.decoder1(e3)
        d2 = self.decoder2(d1)
        d3 = self.decoder3(d2)

        # Final classification
        output = self.classifier(d3)
        return output

#
# ---> Training and Evaluation Function <--- #
#
def run_experiment(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # --- DataLoaders ---
        train_dataset = CIFAR10Colorization(train=True)
        val_dataset = CIFAR10Colorization(train=False)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

        # --- Model, Loss, Optimizer ---
        model = ColorizationNet(nf=config.nf).to(device)
        criterion = nn.CrossEntropyLoss()
        if config.optimizer == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        else:
            optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)
        
        wandb.watch(model, criterion, log="all", log_freq=100)

        # --- Training Loop ---
        print("Starting training...")
        for epoch in range(config.epochs):
            model.train()
            train_loss = 0.0
            for gray_imgs, target_maps in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Train]"):
                gray_imgs, target_maps = gray_imgs.to(device), target_maps.to(device)

                optimizer.zero_grad()
                outputs = model(gray_imgs)
                loss = criterion(outputs, target_maps)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            # --- Validation Step ---
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for gray_imgs, target_maps in tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Val]"):
                    gray_imgs, target_maps = gray_imgs.to(device), target_maps.to(device)
                    outputs = model(gray_imgs)
                    loss = criterion(outputs, target_maps)
                    val_loss += loss.item()
            
            # --- Logging ---
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss / len(train_loader),
                "val_loss": val_loss / len(val_loader)
            })

        print("Training finished.")
        # Log final visualizations
        visualize_results(model, val_loader, "validation_examples")

#
# ---> Visualization Helper <--- #
#
def class_map_to_rgb(class_map, centroids):
    """Converts a map of class indices to an RGB image."""
    rgb_image = centroids[class_map.cpu().numpy()]
    return rgb_image.astype(np.uint8)

def visualize_results(model, loader, log_key):
    """Logs a table of visualization examples to wandb."""
    model.eval()
    table = wandb.Table(columns=["ID", "Grayscale", "Predicted", "Ground Truth"])
    
    with torch.no_grad():
        # Get one batch of images
        gray_imgs, target_maps = next(iter(loader))
        gray_imgs, target_maps = gray_imgs.to(device), target_maps.to(device)
        
        # Get model predictions
        outputs = model(gray_imgs)
        pred_maps = torch.argmax(outputs, dim=1)

        # Add up to 10 examples to the table
        for i in range(min(10, len(gray_imgs))):
            gray_pil = transforms.ToPILImage()(gray_imgs[i].cpu())
            pred_rgb = class_map_to_rgb(pred_maps[i], color_centroids)
            true_rgb = class_map_to_rgb(target_maps[i], color_centroids)

            table.add_data(
                i,
                wandb.Image(gray_pil),
                wandb.Image(pred_rgb),
                wandb.Image(true_rgb)
            )
            
    wandb.log({log_key: table})
    print(f"Logged visualization table to wandb under '{log_key}'.")

#
# ---> Main Execution and Sweep Setup <--- #
#
if __name__ == '__main__':
    # -- To run a single experiment --
    # 1. Define your config
    default_config = {
        "learning_rate": 1e-3,
        "optimizer": "adam",
        "epochs": 25,
        "batch_size": 128,
        "nf": 32 # Number of filters
    }
    # 2. Run the experiment
    # run_experiment(default_config)


    # -- To run a hyperparameter sweep --
    # 1. Define the sweep configuration
    sweep_config = {
        'method': 'random', # 'grid', 'random', 'bayes'
        'metric': {
            'name': 'val_loss',
            'goal': 'minimize'   
        },
        'parameters': {
            'epochs': {'value': 25},
            'batch_size': {'values': [64, 128]},
            'learning_rate': {'distribution': 'uniform', 'min': 1e-4, 'max': 1e-2},
            'optimizer': {'values': ['adam', 'sgd']},
            'nf': {'values': [16, 32, 64]}
        }
    }

    # 2. Initialize the sweep
    sweep_id = wandb.sweep(sweep_config, project="smai-a4-q2-colorization")

    # 3. Start the agent
    wandb.agent(sweep_id, function=run_experiment, count=20) # 'count' is the number of runs


Loading color centroids from data/color_centroids.npy
Create sweep with ID: im3oifyh
Sweep URL: https://wandb.ai/sudersha-sarraf-iiit-hyderabad/smai-a4-q2-colorization/sweeps/im3oifyh


wandb: Agent Starting Run: rq5f7cnb with config:
wandb: 	batch_size: 128
wandb: 	epochs: 25
wandb: 	learning_rate: 0.009311856769146924
wandb: 	nf: 64
wandb: 	optimizer: adam
wandb: Currently logged in as: sudersha-sarraf (sudersha-sarraf-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 170M/170M [00:34<00:00, 4.92MB/s] 


Starting training...


Epoch 25/25 [Val]: 100%|██████████| 79/79 [00:02<00:00, 26.90it/s]

...

Starting training...


Epoch 25/25 [Val]: 100%|██████████| 157/157 [00:03<00:00, 52.19it/s]

Training finished.


Logged visualization table to wandb under 'validation_examples'.


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
train_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▂▂▂▁▁▁▁
epoch,24
train_loss,1.66658
val_loss,1.63814
